In [23]:
import pandas as pd #importing pandas for processing dataset.
import numpy as np # importing numpy for array processing.
import tensorflow as tf # importing tensorflow
from tensorflow.python.data import Dataset # importing Dataset module for preparing dataset.
from sklearn import metrics # importing metrics from sklearn for calculating loss.
import math # import.puing math for some mathamatical operations.

In [55]:
insurance_dataframe = pd.read_csv('/home/pranav/Desktop/workshops-master/TensorFlow study jam/regression.csv', sep=',') #reading the csv dataset to a pandas dataframe
insurance_dataframe = insurance_dataframe.reindex(np.random.permutation(insurance_dataframe.index)) #randomizing the dataset
# print(insurance_dataframe.describe()) # Printing a quick summary of our dataset, like mean, min, max..etc.
print(insurance_dataframe)


      X      Y
1    19   46.2
29    4   38.1
24   29  103.9
45   41   73.4
2    13   15.7
15    2    6.6
35   11   57.2
57   24  137.9
51   13   93.0
8    45  214.0
42   41  181.3
17    6   50.9
22    9   52.1
59   31  209.8
39   16   59.6
60   14   95.5
18    3    4.4
38    4   12.6
50   17  142.1
49    3   39.9
14    7   48.8
40   13   89.9
21    9   48.7
7    14   77.5
62   26  187.5
3   124  422.2
34   22  161.5
41   60  202.4
13   23   39.6
48    8   76.1
..  ...    ...
58    9   87.4
37   12   58.1
6    23   56.9
31   25   69.2
53   15   32.1
32    6   14.6
55   29  133.3
19   23  113.0
4    40  119.4
23    3   13.2
12   11   23.5
25    7   77.5
36   61  217.6
56   30  194.5
46   11   21.3
11   48  248.1
0   108  392.5
26    4   11.8
43   37  152.8
30    0    0.0
47   27   92.6
44   55  162.8
33    5   40.3
52   13   31.9
20    6   14.8
9    10   65.3
54    8   55.6
61   53  244.6
5    57  170.9
10    5   20.9

[63 rows x 2 columns]


In [52]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """
    Input function for batching the dataset. This function return batches of features and targets.
    """
    features = {key:np.array(value) for key, value in dict(features).items()}
    
    ds = Dataset.from_tensor_slices((features, targets))
    ds = ds.batch(batch_size).repeat(num_epochs) #constructing a dataset object from our data, and then break our data into batches of batch_size, to be repeated for the specified number of epochs.
    if shuffle:
        # if shuffle is true, randomize the dataset.
        ds = ds.shuffle(buffer_size=100)

    features, labels = ds.make_one_shot_iterator().get_next() # getting the current batch of feature and targets
    return features, labels


In [83]:
def train_model(learning_rate, steps, batch_size, input_feature="X"):
    """
    Method for training the model. Learning rate, batch size and input features are the parameters. 
    """
    periods = 10
    steps_per_period = steps / periods

    my_feature = input_feature
    my_feature_data = insurance_dataframe[[my_feature]]


    my_label = "Y"
    targets = insurance_dataframe[my_label]

    # Create feature columns.
    feature_columns = [tf.feature_column.numeric_column(my_feature)]

    # Create input functions.
    training_input_fn = lambda:my_input_fn(my_feature_data, targets, batch_size=batch_size) #input function training 
    prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False) # input function for prediction

    # Create a linear regressor object.
    my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    # gradient descent optimise
    my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
    linear_regressor = tf.estimator.LinearRegressor(feature_columns=feature_columns,optimizer=my_optimizer)


    # Train the model, but do so inside a loop so that we can periodically assess loss metrics.
    print("Training model...")
    print("Adam optimzers (on training data):")
    root_mean_squared_errors = []
    for period in range (0, periods):
        # Train the model, starting from the prior state.
        linear_regressor.train(input_fn=training_input_fn, steps=steps_per_period)
        # Take a break and compute predictions.
        predictions = linear_regressor.predict(input_fn=prediction_input_fn)
        predictions = np.array([item['predictions'][0] for item in predictions])

        # Compute loss.
        root_mean_squared_error = math.sqrt(
            metrics.mean_squared_error(predictions, targets))
        # Occasionally print the current loss.
        print("  period %02d : %0.2f" % (period, root_mean_squared_error))
        # Add the loss metrics from this period to our list.
        root_mean_squared_errors.append(root_mean_squared_error)


    print("Model training finished! ")
    print("Final error (on training data): %0.2f" % root_mean_squared_error)
    return linear_regressor





In [84]:
linear_regressor = train_model(learning_rate=0.0001, steps=15000, batch_size=8) # training the model


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp70s_i5kx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5a1e19bf98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training model...
Adam optimzers (on training data):
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp

INFO:tensorflow:global_step/sec: 1583.86
INFO:tensorflow:loss = 11567.781, step = 5001 (0.063 sec)
INFO:tensorflow:global_step/sec: 1595.25
INFO:tensorflow:loss = 11600.3545, step = 5101 (0.062 sec)
INFO:tensorflow:global_step/sec: 1780.96
INFO:tensorflow:loss = 15544.7, step = 5201 (0.056 sec)
INFO:tensorflow:global_step/sec: 1644.31
INFO:tensorflow:loss = 5645.694, step = 5301 (0.061 sec)
INFO:tensorflow:global_step/sec: 1773.15
INFO:tensorflow:loss = 13907.098, step = 5401 (0.056 sec)
INFO:tensorflow:global_step/sec: 1666.14
INFO:tensorflow:loss = 9020.406, step = 5501 (0.060 sec)
INFO:tensorflow:global_step/sec: 1716.33
INFO:tensorflow:loss = 11518.413, step = 5601 (0.058 sec)
INFO:tensorflow:global_step/sec: 1763.39
INFO:tensorflow:loss = 15094.982, step = 5701 (0.057 sec)
INFO:tensorflow:global_step/sec: 1763.54
INFO:tensorflow:loss = 14539.402, step = 5801 (0.057 sec)
INFO:tensorflow:global_step/sec: 1770.61
INFO:tensorflow:loss = 9249.501, step = 5901 (0.057 sec)
INFO:tensorflo

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp70s_i5kx/model.ckpt-10500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10501 into /tmp/tmp70s_i5kx/model.ckpt.
INFO:tensorflow:loss = 20759.404, step = 10501
INFO:tensorflow:global_step/sec: 907.742
INFO:tensorflow:loss = 8958.73, step = 10601 (0.111 sec)
INFO:tensorflow:global_step/sec: 1414.56
INFO:tensorflow:loss = 11652.202, step = 10701 (0.070 sec)
INFO:tensorflow:global_step/sec: 1446.38
INFO:tensorflow:loss = 11373.284, step = 10801 (0.069 sec)
INFO:tensorflow:global_step/sec: 1489.14
INFO:tensorflow:loss = 8605.389, step = 10901 (0.067 sec)
INFO:tensorflow:global_step/sec: 1222.96
INFO:tensorflow:loss = 5606.0205, step = 11001 (0.084 sec)
INFO:tensorflow:global_step/sec: 1239.94
INFO:tensorflow:loss = 11919.473, step = 11101 (0.078 sec)
INFO:tensorflow:global_step/sec: 1472.37
INFO:tensorflow:loss = 20860.537, step =

In [85]:
# new_test = np.array(insurance_dataframe)
new_test_val = np.array(insurance_dataframe)
new_test_val = new_test_val[1:10:,:,]
# new_df = insurance_dataframe['Y']
print(new_test_val)
# print(new_test_val.shape)
# if ((np.array(insurance_dataframe['X'])).all==(new_test_val).all):
#     print("yes")

[[  4.   38.1]
 [ 29.  103.9]
 [ 41.   73.4]
 [ 13.   15.7]
 [  2.    6.6]
 [ 11.   57.2]
 [ 24.  137.9]
 [ 13.   93. ]
 [ 45.  214. ]]


In [86]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": np.array(insurance_dataframe['X'])},
    y=np.array(insurance_dataframe['Y']),
    num_epochs=1,
    shuffle=False)

In [88]:
evals = linear_regressor.evaluate(input_fn=test_input_fn,steps=None)

    
    
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-11-18:23:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp70s_i5kx/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-11-18:23:14
INFO:tensorflow:Saving dict for global step 15000: average_loss = 1372.1709, global_step = 15000, loss = 86446.766
